# Direct Preference Optimization (DPO) Training with SageMaker

This notebook demonstrates how to use the **DPOTrainer** to fine-tune large language models using Direct Preference Optimization (DPO). DPO is a technique that trains models to align with human preferences by learning from preference data without requiring a separate reward model.

## What is DPO?

Direct Preference Optimization (DPO) is a method for training language models to follow human preferences. Unlike traditional RLHF (Reinforcement Learning from Human Feedback), DPO directly optimizes the model using preference pairs without needing a reward model.

**Key Benefits:**
- Simpler than RLHF - no reward model required
- More stable training process
- Direct optimization on preference data
- Works with LoRA for efficient fine-tuning

## Workflow Overview

1. **Prepare Preference Dataset**: Upload preference data in JSONL format
2. **Register Dataset**: Create a SageMaker AI Registry dataset
3. **Configure DPO Trainer**: Set up model, training parameters, and resources
4. **Execute Training**: Run the DPO fine-tuning job
5. **Track Results**: Monitor training with MLflow integration

## Step 1: Prepare and Register Preference Dataset

DPO requires preference data in a specific format where each example contains:
- **prompt**: The input text
- **chosen**: The preferred response
- **rejected**: The less preferred response

The dataset should be in JSONL format with each line containing one preference example.

In [1]:
from sagemaker.ai_registry.dataset import DataSet
from sagemaker.ai_registry.dataset_utils import CustomizationTechnique

'''# Upload dataset to S3
import boto3
s3 = boto3.client('s3')
s3.upload_file(
    './dpo-preference_dataset_train_256.jsonl',
    'nova-mlflow-us-west-2',
    'dataset/preference_dataset_train_256.jsonl'
)'''

# Register dataset in SageMaker AI Registry
# This creates a versioned dataset that can be referenced by ARN
'''dataset = DataSet.create(
    name="demo-nargokul-6", 
    data_location="s3://nova-mlflow-us-west-2/dataset/preference_dataset_train_256.jsonl", 
    customization_technique=CustomizationTechnique.DPO, 
    wait=True
)

print(f"Dataset ARN: {dataset.arn}")'''

[12/01/25 13:29:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=972233;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=418127;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=586988;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=34773;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


'dataset = DataSet.create(\n    name="demo-nargokul-6", \n    data_location="s3://nova-mlflow-us-west-2/dataset/preference_dataset_train_256.jsonl", \n    customization_technique=CustomizationTechnique.DPO, \n    wait=True\n)\n\nprint(f"Dataset ARN: {dataset.arn}")'

## Step 2: Configure and Execute DPO Training

The **DPOTrainer** provides a high-level interface for DPO fine-tuning with the following key features:

### Key Parameters:
- **model**: Base model to fine-tune (from SageMaker Hub)
- **training_type**: Fine-tuning method (LoRA recommended for efficiency)
- **training_dataset**: ARN of the registered preference dataset
- **model_package_group_name**: Where to store the fine-tuned model
- **mlflow_resource_arn**: MLflow tracking server for experiment logging

### Training Features:
- **Serverless Training**: Automatically managed compute resources
- **LoRA Integration**: Parameter-efficient fine-tuning
- **MLflow Tracking**: Automatic experiment and metrics logging
- **Model Versioning**: Automatic model package creation

In [1]:
import random
#! ada credentials update --provider=isengard --account=052150106756 --role=Admin --profile=default --once
#! aws configure set region  us-west-2

from sagemaker.train.dpo_trainer import DPOTrainer
from sagemaker.train.common import TrainingType



[12/01/25 13:40:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=467839;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=684274;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

[12/01/25 13:40:17] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=535804;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=730749;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


In [2]:
# Create DPOTrainer instance with comprehensive configuration
trainer = DPOTrainer(
    # Base model from SageMaker Hub
    model="meta-textgeneration-llama-3-2-1b-instruct",
    
    # Use LoRA for efficient fine-tuning
    training_type=TrainingType.LORA,
    
    # Model versioning and storage
    model_package_group_name="sdk-test-finetuned-models",
    
    # MLflow experiment tracking
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/ashwpat-test",
    
    # Training data (from Step 1)
    training_dataset="s3://mc-flows-sdk-testing/input_data/dpo/preference_dataset_train_256.jsonl",
    
    # Output configuration
    s3_output_path="s3://mc-flows-sdk-testing/output/",
    
    # IAM role for training job
    #role="arn:aws:iam::052150106756:role/Admin",
    
    # Unique job name
    base_job_name=f"dpo-llama-{random.randint(1, 1000)}",
    accept_eula=True
)

# Customize training hyperparameters
# DPO-specific parameters are automatically loaded from the model's recipe
trainer.hyperparameters.max_epochs = 1  # Quick training for demo

print("Starting DPO training job...")
print(f"Job name: {trainer.base_job_name}")
print(f"Base model: {trainer._model_name}")

# Execute training with monitoring
training_job = trainer.train(wait=True)

print(f"Training completed! Job ARN: {training_job.training_job_arn}")

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      dpo-llama-695-20251201134040                                      │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          216.7s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             0.7s                      │
│   ✓    Pending           Preparing the instances for           24.0s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        10.5s                     │
│   ✓    Training          Training image download completed.    165.9s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    12.9s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

Training completed! Job ARN: arn:aws:sagemaker:us-west-2:729646638167:training-job/dpo-llama-695-20251201134040


In [3]:
print(training_job)

training_job_name='dpo-llama-45-20251129130016' training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/dpo-llama-45-20251129130016' processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1525d40e0> tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1525d40e0> labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1525d40e0> auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1525d40e0> model_artifacts=ModelArtifacts(s3_model_artifacts='s3://nova-mlflow-us-west-2/output/dpo-llama-45-20251129130016/output/model') training_job_output=TrainingJobOutput(s3_training_job_output='s3://nova-mlflow-us-west-2/output/dpo-llama-45-20251129130016/output/output') training_job_status='Completed' secondary_status='Completed' failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1525d40e0> hyper_parameters={'adam_beta': '0.01', 'global_batch_size': '16', 'learning_rate': '0.0001', 'lr_warmup_ratio': '0.1', 'max_epochs

In [10]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="dpo-llama-557-20251125140706")
print(response)

training_job_name='dpo-llama-557-20251125140706' training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/dpo-llama-557-20251125140706' processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> model_artifacts=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> training_job_output=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> training_job_status='InProgress' secondary_status='Training' failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x17e07d4f0> hyper_parameters={'adam_beta': '0.01', 'global_batch_size': '16', 'learning_rate': '0.0001', 'lr_warmup_ratio': '0.1', 'max_epochs': '1', 'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct', 'name': 'example-name-2ke5m', 'train_val

## Next Steps

After training completes, you can:

1. **Deploy the Model**: Use `ModelBuilder` to deploy the fine-tuned model
2. **Evaluate Performance**: Compare responses from base vs fine-tuned model
3. **Monitor Metrics**: Review training metrics in MLflow
4. **Iterate**: Adjust hyperparameters and retrain if needed

### Example Deployment:
```python
from sagemaker.serve import ModelBuilder

# Deploy the fine-tuned model
model_builder = ModelBuilder(model=training_job)
model_builder.build(role_arn="arn:aws:iam::account:role/SageMakerRole")
endpoint = model_builder.deploy(endpoint_name="dpo-finetuned-llama")
```

The fine-tuned model will now generate responses that better align with the preferences in your training data.